In [7]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [3]:
train_df = pd.read_csv('input/train.csv')
# train_df = pd.read_csv('input/train_min.csv')
test_df = pd.read_csv('input/test.csv')

X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [4]:
n_splits = 10
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [5]:
params = {
         'num_leaves': 8,
         'min_data_in_leaf': 42,
         'objective': 'binary',
         'max_depth': 16,
         'learning_rate': 0.0123,
         'boosting': 'gbdt',
         'bagging_freq': 5,
         'bagging_fraction': 0.8,
         'feature_fraction': 0.8201,
         'bagging_seed': 11,
         'reg_alpha': 1.728910519108444,
         'reg_lambda': 4.9847051755586085,
         'random_state': 42,
         'metric': 'auc',
         'verbosity': -1,
         'subsample': 0.81,
         'min_gain_to_split': 0.01077313523861969,
         'min_child_weight': 19.428902804238373,
         'num_threads': 4}

In [8]:
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    model = lgb.train(params,train_data,num_boost_round=20000,
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 200)
    
    prediction += model.predict(X_test, num_iteration=model.best_iteration)/n_splits

Fold 0 started at Thu Feb 28 22:56:50 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.826269	valid_1's auc: 0.815427
[600]	training's auc: 0.860084	valid_1's auc: 0.846328
[900]	training's auc: 0.8784	valid_1's auc: 0.861877
[1200]	training's auc: 0.889833	valid_1's auc: 0.87074
[1500]	training's auc: 0.897955	valid_1's auc: 0.877081
[1800]	training's auc: 0.904016	valid_1's auc: 0.881368
[2100]	training's auc: 0.908657	valid_1's auc: 0.884588
[2400]	training's auc: 0.912626	valid_1's auc: 0.88718
[2700]	training's auc: 0.916	valid_1's auc: 0.889296
[3000]	training's auc: 0.918762	valid_1's auc: 0.891107
[3300]	training's auc: 0.921155	valid_1's auc: 0.89269
[3600]	training's auc: 0.92319	valid_1's auc: 0.893783
[3900]	training's auc: 0.924952	valid_1's auc: 0.894802
[4200]	training's auc: 0.926543	valid_1's auc: 0.895664
[4500]	training's auc: 0.927982	valid_1's auc: 0.896364
[4800]	training's auc: 0.929386	valid_1's auc: 0.896952
[5100]	tra

[300]	training's auc: 0.825729	valid_1's auc: 0.813909
[600]	training's auc: 0.860663	valid_1's auc: 0.84554
[900]	training's auc: 0.878587	valid_1's auc: 0.860628
[1200]	training's auc: 0.889715	valid_1's auc: 0.870377
[1500]	training's auc: 0.897608	valid_1's auc: 0.877308
[1800]	training's auc: 0.903874	valid_1's auc: 0.882645
[2100]	training's auc: 0.908483	valid_1's auc: 0.886513
[2400]	training's auc: 0.912234	valid_1's auc: 0.889394
[2700]	training's auc: 0.915625	valid_1's auc: 0.891709
[3000]	training's auc: 0.918457	valid_1's auc: 0.893709
[3300]	training's auc: 0.920838	valid_1's auc: 0.895316
[3600]	training's auc: 0.922816	valid_1's auc: 0.896451
[3900]	training's auc: 0.924519	valid_1's auc: 0.897611
[4200]	training's auc: 0.926111	valid_1's auc: 0.898677
[4500]	training's auc: 0.927535	valid_1's auc: 0.89916
[4800]	training's auc: 0.928924	valid_1's auc: 0.899787
[5100]	training's auc: 0.930156	valid_1's auc: 0.900251
[5400]	training's auc: 0.931337	valid_1's auc: 0.9005

In [9]:
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = prediction
sub.to_csv("submission.csv", index=False)